# Goal

* Primer design for clade of interest

# Var

In [19]:
base_dir = '/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/'
clade = 'Oscillibacter'
domain = 'Bacteria'
taxid = 459786

# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(LeyLabRMisc)

In [5]:
df.dims()
work_dir = file.path(base_dir, clade)
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Oscillibacter 


# Genome download

* Downloading genomes from NCBI

```
OUTDIR=/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/
mkdir -p $OUTDIR
ncbi-genome-download -p 12 -s genbank -F fasta --genera Oscillibacter -o $OUTDIR bacteria
```

# Genome quality

* Filtering genomes by quality

In [6]:
D = file.path(base_dir, clade, 'genbank')
files = list_files(D, '.fna.gz')
samps = data.frame(Name = files %>% as.character %>% basename,
                   Fasta = files,
                   Domain = 'Bacteria',
                   Taxid = taxid) %>%
    mutate(Name = gsub('\\.fna\\.gz$', '', Name),
           Fasta = gsub('/+', '/', Fasta))
samps

# writing file
outfile = file.path(D, 'samples.txt')
write_table(samps, outfile)

Name,Fasta,Domain,Taxid
<chr>,<chr>,<fct>,<dbl>
GCA_000283575.1_ASM28357v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/genbank/bacteria/GCA_000283575.1/GCA_000283575.1_ASM28357v1_genomic.fna.gz,Bacteria,459786
GCA_000307265.1_ASM30726v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/genbank/bacteria/GCA_000307265.1/GCA_000307265.1_ASM30726v1_genomic.fna.gz,Bacteria,459786
⋮,⋮,⋮,⋮
GCA_015052085.1_ASM1505208v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/genbank/bacteria/GCA_015052085.1/GCA_015052085.1_ASM1505208v1_genomic.fna.gz,Bacteria,459786
GCA_900115635.1_IMG-taxon_2623620509_annotated_assembly_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/genbank/bacteria/GCA_900115635.1/GCA_900115635.1_IMG-taxon_2623620509_annotated_assembly_genomic.fna.gz,Bacteria,459786


File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Oscillibacter/genbank/samples.txt 


### LLG

#### Config

In [7]:
cat_file(file.path(work_dir, 'LLG', 'config.yaml'))

# table with genome --> fasta_file information
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/genbank/samples.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/LLG/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

# batch processing of genomes for certain steps
## increase to better parallelize
batches: 2 

# Domain of genomes ('Archaea' or 'Bacteria)
## Use "Skip" if provided as a "Domain" column in the genome table
Domain: Skip

# software parameters
# Use "Skip" to skip any of these steps. If no params for rule, use ""
# dRep MAGs are not further analyzed, but you can de-rep & then use the de-rep genome table as input.
params:
  ionice: -c 3
  # assembly assessment
  seqkit: ""
  quast: Skip #""
  multiqc_on_quast: "" 
  checkm: ""
  # de-replication (requires checkm)
  drep: -

#### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llg
$ screen -L -S llg-osc ./snakemake_sge.sh /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/LLG/llg.yaml 20 -F
```

### Samples table of high-quality genomes

In [8]:
# checkM summary
checkm = file.path(work_dir, 'LLG', 'checkM', 'checkm_qa_summary.tsv') %>%
    read.delim(sep='\t') 
checkm

Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,Strain.heterogeneity,Genome.size..bp.,X..ambiguous.bases,⋯,X0,X1,X2,X3,X4,X5.,assembly.Id,assembler.Id,taxon.Id,File
<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<lgl>,<fct>
GCA_000283575.1_ASM28357v1_genomic,o__Clostridiales (UID1212),172,263,149,98.99,1.01,0,4470622,0,⋯,2,259,2,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Oscillibacter|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.1,markers_qa_summary.tsv.1,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.1
GCA_000403435.2_Osci_bact_1-3_V1_genomic,o__Clostridiales (UID1212),172,263,149,99.33,1.45,0,4467686,37481,⋯,1,259,3,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Oscillibacter|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.2,markers_qa_summary.tsv.2,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_014799915.1_ASM1479991v1_genomic,o__Clostridiales (UID1212),172,263,149,87.74,2.35,50.00,2875254,19077,⋯,50,209,4,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Oscillibacter|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.18,markers_qa_summary.tsv.18,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.18
GCA_015052085.1_ASM1505208v1_genomic,o__Clostridiales (UID1212),172,263,149,89.56,11.63,82.61,2465477,0,⋯,45,195,23,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Oscillibacter|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.19,markers_qa_summary.tsv.19,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.19


In [9]:
# dRep summary
drep = file.path(work_dir, 'LLG', 'drep', 'checkm_markers_qa_summary.tsv') %>%
    read.delim(sep='\t') %>%
    mutate(Bin.Id = gsub('.+/', '', genome),
           Bin.Id = gsub('\\.fna$', '', Bin.Id))
drep

genome,completeness,contamination,Bin.Id
<fct>,<dbl>,<dbl>,<chr>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000283575.1_ASM28357v1_genomic.fna,98.99,1.01,GCA_000283575.1_ASM28357v1_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000403435.2_Osci_bact_1-3_V1_genomic.fna,99.33,1.45,GCA_000403435.2_Osci_bact_1-3_V1_genomic
⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_014799915.1_ASM1479991v1_genomic.fna,87.74,2.35,GCA_014799915.1_ASM1479991v1_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_015052085.1_ASM1505208v1_genomic.fna,89.56,11.63,GCA_015052085.1_ASM1505208v1_genomic


In [10]:
# de-replicated genomes
drep_gen = file.path(work_dir, 'LLG', 'drep', 'dereplicated_genomes.tsv') %>%
    read.delim(sep='\t')
drep_gen

Name,Fasta
<fct>,<fct>
GCA_000765235.1_ASM76523v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/drep/drep/dereplicated_genomes/GCA_000765235.1_ASM76523v1_genomic.fna
GCA_003525445.1_ASM352544v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/drep/drep/dereplicated_genomes/GCA_003525445.1_ASM352544v1_genomic.fna
⋮,⋮
GCA_001916835.1_ASM191683v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/drep/drep/dereplicated_genomes/GCA_001916835.1_ASM191683v1_genomic.fna
GCA_014799925.1_ASM1479992v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Oscillibacter/LLG_output/drep/drep/dereplicated_genomes/GCA_014799925.1_ASM1479992v1_genomic.fna


In [11]:
# GTDBTk summary
tax = file.path(work_dir, 'LLG', 'gtdbtk', 'gtdbtk_bac_summary.tsv') %>%
    read.delim(, sep='\t') %>%
    separate(classification, 
             c('Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species'),
             sep=';') %>%
    select(-note, -classification_method, -pplacer_taxonomy,
           -other_related_references.genome_id.species_name.radius.ANI.AF.)
tax

user_genome,Domain,Phylum,Class,Order,Family,Genus,Species,fastani_reference,fastani_reference_radius,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
GCA_000283575.1_ASM28357v1_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Oscillospiraceae,g__Oscillibacter,s__Oscillibacter valericigenes,GCF_000283575.1,95.0,⋯,1.0,GCF_000283575.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter valericigenes,100.0,1.0,95.38,11,N/A,N/A
GCA_000403435.2_Osci_bact_1-3_V1_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Oscillospiraceae,g__Oscillibacter,s__Oscillibacter sp000403435,GCF_000403435.2,95.0,⋯,0.99,GCF_000403435.2,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter sp000403435,99.99,0.99,95.67,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_014799915.1_ASM1479991v1_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Oscillospiraceae,g__,s__,N/A,N/A,⋯,N/A,N/A,N/A,N/A,N/A,N/A,79.35,11,0.9173648727242681,N/A
GCA_015052085.1_ASM1505208v1_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Oscillospiraceae,g__Oscillibacter,s__Oscillibacter welbionis,GCF_005121165.1,95.0,⋯,0.88,GCF_005121165.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter welbionis,98.68,0.88,76.73,11,N/A,N/A


In [12]:
# checking overlap
cat('-- drep --\n')
overlap(basename(as.character(drep_gen$Fasta)), 
        basename(as.character(drep$genome)))
cat('-- checkm --\n')
overlap(drep$Bin.Id, checkm$Bin.Id)
cat('-- gtdbtk --\n')
overlap(drep$Bin.Id, tax$user_genome)

-- drep --
intersect(x,y): 16 
setdiff(x,y): 0 
setdiff(y,x): 23 
union(x,y): 39 
-- checkm --
intersect(x,y): 39 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 39 
-- gtdbtk --
intersect(x,y): 39 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 39 


In [13]:
# joining based on Bin.Id
drep = drep %>%
    inner_join(checkm, c('Bin.Id')) %>%
    mutate(GEN = genome %>% as.character %>% basename) %>%
    inner_join(drep_gen %>% mutate(GEN = Fasta %>% as.character %>% basename),
               by=c('GEN')) %>%
    inner_join(tax, c('Bin.Id'='user_genome')) #%>%
drep

genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000283575.1_ASM28357v1_genomic.fna,98.99,1.01,GCA_000283575.1_ASM28357v1_genomic,o__Clostridiales (UID1212),172,263,149,98.99,1.01,⋯,1.0,GCF_000283575.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter valericigenes,100.0,1.0,95.38,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000403435.2_Osci_bact_1-3_V1_genomic.fna,99.33,1.45,GCA_000403435.2_Osci_bact_1-3_V1_genomic,o__Clostridiales (UID1212),172,263,149,99.33,1.45,⋯,0.99,GCF_000403435.2,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter sp000403435,99.99,0.99,95.67,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_003497655.1_ASM349765v1_genomic.fna,91.14,0.00,GCA_003497655.1_ASM349765v1_genomic,o__Clostridiales (UID1212),172,263,149,91.14,0.00,⋯,0.82,GCF_000765235.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__ER4;s__ER4 sp000765235,96.71,0.82,91.90,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_009774015.1_ASM977401v1_genomic.fna,91.03,3.68,GCA_009774015.1_ASM977401v1_genomic,o__Clostridiales (UID1212),172,263,149,91.03,3.68,⋯,N/A,GCF_000403435.2,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter sp000403435,82.6,0.59,83.51,11,0.9623815263719147,N/A


In [14]:
# filtering by quality
hq_genomes = drep %>%
    filter(completeness >= 90,
           contamination < 5,
           Strain.heterogeneity < 50)
hq_genomes

genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000283575.1_ASM28357v1_genomic.fna,98.99,1.01,GCA_000283575.1_ASM28357v1_genomic,o__Clostridiales (UID1212),172,263,149,98.99,1.01,⋯,1.0,GCF_000283575.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter valericigenes,100.0,1.0,95.38,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000403435.2_Osci_bact_1-3_V1_genomic.fna,99.33,1.45,GCA_000403435.2_Osci_bact_1-3_V1_genomic,o__Clostridiales (UID1212),172,263,149,99.33,1.45,⋯,0.99,GCF_000403435.2,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter sp000403435,99.99,0.99,95.67,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_003497655.1_ASM349765v1_genomic.fna,91.14,0.00,GCA_003497655.1_ASM349765v1_genomic,o__Clostridiales (UID1212),172,263,149,91.14,0.00,⋯,0.82,GCF_000765235.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__ER4;s__ER4 sp000765235,96.71,0.82,91.90,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_009774015.1_ASM977401v1_genomic.fna,91.03,3.68,GCA_009774015.1_ASM977401v1_genomic,o__Clostridiales (UID1212),172,263,149,91.03,3.68,⋯,N/A,GCF_000403435.2,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Oscillibacter;s__Oscillibacter sp000403435,82.6,0.59,83.51,11,0.9623815263719147,N/A


In [15]:
# summarizing the taxonomy
df.dims(20)
hq_genomes %>%
    group_by(Family, Genus) %>%
    summarize(n_genomes = n(), .groups='drop')
df.dims()

Family,Genus,n_genomes
<chr>,<chr>,<int>
f__Oscillospiraceae,g__,1
f__Oscillospiraceae,g__CAG-83,1
f__Oscillospiraceae,g__ER4,3
f__Oscillospiraceae,g__Oscillibacter,9


In [20]:
# writing samples table for LLPRIMER
outfile = file.path(work_dir, 'samples_genomes_hq.txt')
hq_genomes %>%
    select(Bin.Id, Fasta) %>%
    rename('Taxon' = Bin.Id) %>%
    mutate(Taxon = gsub('_chromosome.+', '', Taxon),
           Taxon = gsub('_bin_.+', '', Taxon),
           Taxon = gsub('_genomic', '', Taxon),
           Taxon = gsub('_annotated_assembly', '', Taxon),
           Taxid = taxid,
           Domain = domain) %>%
    write_table(outfile)

File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Oscillibacter/samples_genomes_hq.txt 


# Primer design

### Config

In [17]:
F = file.path(work_dir, 'primers', 'config.yaml')
cat_file(F)

#-- I/O --#
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/samples_genomes_hq.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/primers/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

#-- software parameters --#
# See the README for a description
params:
  ionice: -c 3
  cgp:
    prokka: ""
    mmseqs:
      method: cluster    # or linclust (faster)
      run: --min-seq-id 0.9 -c 0.8
    vsearch: --id 0.94
    core_genes:
      cds: --perc-genomes-cds 100 --copies-per-genome-cds 1 --max-clusters-cds 10 #500
      rRNA: --perc-genomes-rrna 100 --copies-per-genome-rrna 10 --max-clusters-rrna 500
    align:
      method: linsi
      params: --auto --maxiterate 1000
    primer3:
      consensus: --consensus-threshold 0.34
      number: --num-raw-primers 1000 --num-final-primers 10
 

### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llprimer
$ screen -L -S llprimer-Osc ./snakemake_sge.sh /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Oscillibacter/primers/config.yaml 30 --notemp
```

## Summary

### Primers

In [92]:
primer_info = read.delim(file.path(work_dir, 'primers', 'cgp', 'primers_final_info.tsv'), sep='\t')
primer_info %>% unique_n('primers', primer_set)
primer_info

No. of unique primers: 162 


gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,⋯,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd
<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
cds,1,468,189,189,0,468f,PRIMER_LEFT,AARAACGGYCGYGAYTCYRA,20,⋯,54,74,60.76051,2.848347,50.00000,6.123724,16.30579,21.51507,22.824341,16.40584
cds,1,468,189,189,0,468r,PRIMER_RIGHT,RCCSAGVYGCTCRAARCG,18,⋯,225,243,62.16555,2.574813,64.81481,6.141898,35.16456,27.78734,4.565427,23.59421
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
cds,38,911,204,204,0,911f,PRIMER_LEFT,GGHAAGATGCTSAAYGTSGA,20,⋯,1401,1421,58.39153,1.903507,49.16667,3.435921,21.547320,21.60791,-18.44452,20.099912
cds,38,911,204,204,0,911r,PRIMER_RIGHT,SCGGAARAARAAVGTCARMAG,21,⋯,1584,1605,57.91049,1.924208,46.03175,5.264484,4.451632,11.86112,-24.35675,9.648227


### Gene clusters

In [46]:
# target gene info
gene_annot = read.delim(file.path(work_dir, 'primers', 'cgp', 'core_clusters_info.tsv'), 
                        sep='\t') %>%
    semi_join(primer_info, c('cluster_id')) 
gene_annot %>% distinct(gene_type, cluster_id) %>% nrow %>% cat('No. of clusters', ., '\n')
gene_annot

No. of clusters 26 


gene_type,cluster_id,seq_uuid,seq_orig_name,contig_id,taxon,start,end,score,strand,annotation,cluster_name,clust_id
<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>
cds,1,0d69d3fe0064481cbd2260b6630da5ef,FKNECIIB_01044,AP012044.1,GCA_000283575.1_ASM28357v1,1072042,1072323,.,-,50S ribosomal protein L27,05b02d57263741a486e49b34bde0fa28,1
cds,1,18f7efb1fe36464b80562c6f3fa295b7,MKEBFIFM_00152,KE159703.1,GCA_000403435.2_Osci_bact_1-3_V1,150345,150626,.,+,50S ribosomal protein L27,05b02d57263741a486e49b34bde0fa28,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
cds,38,3bfb278fd5594012b6ce303878008895,MINOPNOJ_01286,DNON01000002.1,GCA_003497655.1_ASM349765v1,9130,11097,.,+,DNA gyrase subunit B,5730334f42ee4013a044d98cfa4e837c,38
cds,38,f51bc6153d974eca99492cceaa1dfaf3,ONMBBDAA_00481,VSMZ01000014.1,GCA_009774015.1_ASM977401v1,18854,20803,.,+,DNA gyrase subunit B,5730334f42ee4013a044d98cfa4e837c,38


### Gene cluster annotations

In [48]:
# non-target cds gene annotations
gene_nontarget = read.delim(file.path(work_dir, 'primers', 'cgp', 'nontarget', 'cds_blastx.tsv'), 
                        sep='\t') %>%
    semi_join(primer_info, c('cluster_id')) 
gene_nontarget

cluster_id,query,subject,subject_name,pident,length,mismatch,qstart,qend,sstart,send,evalue,slen,qlen,sscinames,staxids,pident_rank
<int>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<fct>,<fct>,<int>
1,05b02d57263741a486e49b34bde0fa28,WP_025543765.1,MULTISPECIES: 50S ribosomal protein L27 [Oscillospiraceae],100.000,93,0,1,279,1,93,1.43e-59,93,282,Oscillospiraceae;Oscillibacter valericigenes;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis;Oscillibacter sp. PEA192,216572;351091;1226322;1226323;2093857;2109687,30
1,05b02d57263741a486e49b34bde0fa28,WP_187032256.1,50S ribosomal protein L27 [Dysosmobacter sp. NSJ-60],86.022,93,13,1,279,1,93,1.68e-50,93,282,Dysosmobacter sp. NSJ-60,2763041,2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
38,5730334f42ee4013a044d98cfa4e837c,CDA81968.1,dNA gyrase subunit B [Firmicutes bacterium CAG:176],81.985,655,107,1,1965,1,644,0,644,1968,Firmicutes bacterium CAG:176,1263007,26
38,5730334f42ee4013a044d98cfa4e837c,WP_199648700.1,None,82.290,655,107,1,1965,1,646,0,646,1968,Pseudoflavonifractor sp. AF19-9AC,2292244,22


### Cluster taxonomic uniqueness

In [67]:
# closest seqID
gene_nontarget %>%
    filter(pident_rank == 1) %>%
    .$pident %>% summary_x('seq-ID to closest non-target')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
seq-ID to closest non-target,88.413,92.683,95.868,95.28045,98.462,99.806,4.114,1.68


### Primer quality

In [49]:
# summary
primer_info %>% unique_n('primer sets', primer_set)
primer_info %>% distinct(gene_type, cluster_id) %>% nrow %>% cat('No. of unique clusters:', ., '\n')

No. of unique primer sets: 162 
No. of unique clusters: 26 


In [50]:
# primers per cluster
primer_info %>%
    distinct(cluster_id, primer_set) %>%
    group_by(cluster_id) %>%
    summarize(n_primer_sets = n(), .groups='drop') %>%
    .$n_primer_sets %>% summary_x('primer sets per cluster')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
primer sets per cluster,1,4.25,9,7.115385,10,10,3.62,1.478


In [51]:
# primer quality
primer_info %>% filter(primer_type == 'PRIMER_LEFT') %>% .$amplicon_size_avg %>% summary_x('mean amplicon size')
primer_info %>% .$degeneracy %>% summary_x('degeneracy')
primer_info %>% .$degeneracy_3prime %>% summary_x('degeneracy (3-prime)')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
mean amplicon size,100,106,124,147.6475,194,250,58.267,23.788


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
degeneracy,1,16,24,33.53514,48,128,45.132,18.425


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
degeneracy (3-prime),1,1,2,2.108108,2,6,1.859,0.759


In [73]:
# adding non-target pident info
tmp1 = gene_nontarget %>%
    filter(pident_rank == 1) %>%
    distinct(cluster_id, pident, sscinames) %>%
    mutate(gene_type = 'cds')

primer_info_j = primer_info %>%
    inner_join(tmp1, c('gene_type', 'cluster_id'))
primer_info_j

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,⋯,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
cds,1,468,189,189,0,468f,PRIMER_LEFT,AARAACGGYCGYGAYTCYRA,20,⋯,60.76051,2.848347,50.00000,6.123724,16.30579,21.51507,22.824341,16.40584,91.011,Oscillospiraceae bacterium
cds,1,468,189,189,0,468r,PRIMER_RIGHT,RCCSAGVYGCTCRAARCG,18,⋯,62.16555,2.574813,64.81481,6.141898,35.16456,27.78734,4.565427,23.59421,91.011,Oscillospiraceae bacterium
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
cds,38,911,204,204,0,911f,PRIMER_LEFT,GGHAAGATGCTSAAYGTSGA,20,⋯,58.39153,1.903507,49.16667,3.435921,21.547320,21.60791,-18.44452,20.099912,98.626,Ruminococcaceae bacterium TF06-43
cds,38,911,204,204,0,911r,PRIMER_RIGHT,SCGGAARAARAAVGTCARMAG,21,⋯,57.91049,1.924208,46.03175,5.264484,4.451632,11.86112,-24.35675,9.648227,98.626,Ruminococcaceae bacterium TF06-43


In [82]:
# arrange by "best" primers
primer_info_jf = primer_info_j %>%
    group_by(primer_set) %>%
    mutate(max_degeneracy_3prime = max(degeneracy_3prime),  # max of [Fwd,Rev]
           max_degeneracy = max(degeneracy)) %>%
    ungroup() %>%
    arrange(pident, max_degeneracy_3prime, max_degeneracy) %>%
    head(n=30)

df.dims(30,40)
primer_info_jf
df.dims()

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,degeneracy,degeneracy_3prime,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames,max_degeneracy_3prime,max_degeneracy
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>
cds,24,101,160,160,0,101f,PRIMER_LEFT,TBTTCCGBGAYAAYTCCACS,20,72,2,196,216,59.40978,2.2201857,51.66667,4.859127,23.170159,23.39333,3.8426830,12.757158,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72
cds,24,101,160,160,0,101r,PRIMER_RIGHT,ATSACRTCVGCCATRAASGA,20,48,2,336,356,59.29837,2.1329283,48.33333,4.249183,37.237596,11.36689,-0.2072119,21.774421,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72
cds,24,925,159,159,0,925f,PRIMER_LEFT,TBTTCCGBGAYAAYTCCACS,20,72,2,196,216,59.40978,2.2201857,51.66667,4.859127,23.170159,23.39333,3.8426830,12.757158,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72
cds,24,925,159,159,0,925r,PRIMER_RIGHT,TSACRTCVGCCATRAASGA,19,48,2,336,355,58.81608,2.2386673,50.87719,4.472824,37.237596,11.36689,-0.9210938,21.070817,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72
cds,24,252,205,205,0,252f,PRIMER_LEFT,ACSGCSAACATGGTBTCSTT,20,24,2,321,341,62.52437,0.8325446,53.33333,2.357023,34.079778,15.71851,8.6676870,8.532017,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,96
cds,24,252,205,205,0,252r,PRIMER_RIGHT,YRTGSAGCATRTCVGCCATR,20,96,2,506,526,60.37872,2.5635953,53.33333,5.527708,39.814057,14.18557,3.8855806,13.889514,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,96
cds,24,852,135,135,0,852f,PRIMER_LEFT,GBTTCTCVTTYGCCTCYGC,19,36,2,223,242,60.45917,2.1671838,59.64912,5.114870,9.565666,16.56822,-52.4259519,8.602116,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,4,48
cds,24,852,135,135,0,852r,PRIMER_RIGHT,CGATSACRTCVGCCATRAAS,20,48,4,338,358,59.09106,2.0362332,53.33333,4.249183,34.242766,16.18458,5.7775265,14.765420,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,4,48
cds,24,962,133,133,0,962f,PRIMER_LEFT,TTCTCVTTYGCCTCYGCST,19,24,4,225,244,61.47875,2.0007686,56.14035,4.472824,29.250102,23.92131,-27.4141990,26.541144,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,4,48


#### w/ gene annotations

In [86]:
# joining
primer_info_jfj = primer_info_jf %>%
    left_join(gene_annot %>% distinct(gene_type, cluster_id, annotation),
              c('gene_type', 'cluster_id'))

df.dims(30,50)
primer_info_jfj
df.dims()

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,degeneracy,degeneracy_3prime,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames,max_degeneracy_3prime,max_degeneracy,annotation
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<fct>
cds,24,101,160,160,0,101f,PRIMER_LEFT,TBTTCCGBGAYAAYTCCACS,20,72,2,196,216,59.40978,2.2201857,51.66667,4.859127,23.170159,23.39333,3.8426830,12.757158,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72,Putative carbamoyltransferase YgeW
cds,24,101,160,160,0,101r,PRIMER_RIGHT,ATSACRTCVGCCATRAASGA,20,48,2,336,356,59.29837,2.1329283,48.33333,4.249183,37.237596,11.36689,-0.2072119,21.774421,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72,Putative carbamoyltransferase YgeW
cds,24,925,159,159,0,925f,PRIMER_LEFT,TBTTCCGBGAYAAYTCCACS,20,72,2,196,216,59.40978,2.2201857,51.66667,4.859127,23.170159,23.39333,3.8426830,12.757158,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72,Putative carbamoyltransferase YgeW
cds,24,925,159,159,0,925r,PRIMER_RIGHT,TSACRTCVGCCATRAASGA,19,48,2,336,355,58.81608,2.2386673,50.87719,4.472824,37.237596,11.36689,-0.9210938,21.070817,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,72,Putative carbamoyltransferase YgeW
cds,24,252,205,205,0,252f,PRIMER_LEFT,ACSGCSAACATGGTBTCSTT,20,24,2,321,341,62.52437,0.8325446,53.33333,2.357023,34.079778,15.71851,8.6676870,8.532017,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,96,Putative carbamoyltransferase YgeW
cds,24,252,205,205,0,252r,PRIMER_RIGHT,YRTGSAGCATRTCVGCCATR,20,96,2,506,526,60.37872,2.5635953,53.33333,5.527708,39.814057,14.18557,3.8855806,13.889514,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,2,96,Putative carbamoyltransferase YgeW
cds,24,852,135,135,0,852f,PRIMER_LEFT,GBTTCTCVTTYGCCTCYGC,19,36,2,223,242,60.45917,2.1671838,59.64912,5.114870,9.565666,16.56822,-52.4259519,8.602116,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,4,48,Putative carbamoyltransferase YgeW
cds,24,852,135,135,0,852r,PRIMER_RIGHT,CGATSACRTCVGCCATRAAS,20,48,4,338,358,59.09106,2.0362332,53.33333,4.249183,34.242766,16.18458,5.7775265,14.765420,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,4,48,Putative carbamoyltransferase YgeW
cds,24,962,133,133,0,962f,PRIMER_LEFT,TTCTCVTTYGCCTCYGCST,19,24,4,225,244,61.47875,2.0007686,56.14035,4.472824,29.250102,23.92131,-27.4141990,26.541144,88.413,Oscillospiraceae;uncultured Blautia sp.;Oscillibacter sp. KLE 1728;Oscillibacter sp. KLE 1745;Dysosmobacter welbionis,4,48,Putative carbamoyltransferase YgeW


#### Writing table

In [88]:
F = file.path(work_dir, 'best_primers.tsv')
write_table(primer_info_jfj, F)

File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Oscillibacter/best_primers.tsv 


# sessionInfo

In [89]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llg/')

LLG
===

Ley Lab Genome analysis pipeline (LLG)

* Version: 0.2.4
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/gtdbtk.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- pigz
- conda-forge::networkx
- bioconda::gtdbtk

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/checkm.yaml <==
channels:
- bioconda
dependencies:
- python=2.7
- pigz
- bioconda::prodigal
- bioconda::pplacer
- bioconda::checkm-genome

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/quast.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::seqkit
- bioconda::quast>=5.0.0

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/sourmash.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::sourmash=2.0.0a4

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/fastqc.yaml <==
channels:
-

In [90]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/')

LLPRIMER

Ley Lab Primer generation pipeline (LLPRIMER)

* Version: 0.3.0
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/pdp.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- python=3.7
- intervaltree
- prodigal
- blast
- bedtools
- mafft
- mummer=3.23
- emboss
- primer3=1.1.4
- biopython<1.78
- pybedtools
- joblib
- tqdm
- openpyxl

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/genes.yaml <==
channels:
- bioconda
dependencies:
- pigz
- python=3
- numpy
- pyfaidx
- bioconda::seqkit
- bioconda::fasta-splitter
- bioconda::vsearch
- bioconda::mmseqs2
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/aln.yaml <==
channels:
- bioconda
- conda-forge
dependencies:
- pigz
- bioconda::revtrans
- bioconda::kalign3
- bioconda::mafft

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/env

In [91]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.6 ggplot2_3.3.1     tidyr_1.1.0       dplyr_1.0.0      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4.6     magrittr_1.5     munsell_0.5.0    tidyselect_1.1.0
 [5] uuid_0.1-4       colorspace_1.4-1 R6_2.4.1         rlang_0.4.6     
 [9] fansi_0.4.1